In [ ]:
#discussion about deployment vs statefulset
    # Deployment is a resource to deploy a stateless application, if using a PVC, all replicas will be using the same Volume and none of it will have its own state.
    # Statefulsets is used for Stateful applications, each replica of the pod will have its own state, and will be using its own Volume.

#Note about statefull vs stateless application in kubernetes:
    # Stateless application is an application which do not depend on outside resource like database
    # Statefull application is an application which depend on other outside resource like database

In [ ]:
#Note about worker component to deploy worker:
    #StatefulSet: is object which use to deploy stateful application
    #Service: define how to access pod of Deployment or StatefulSet
    #PodDisruptionBudget (pod disruption budget): define the minimum pod of Deployment or StatefulSet. Therefore, we can not delete by command if pod equal to minimum limit
    #HorizontalPodAutoscaler: define how to increase number of pods

#Note about horizontal vs vertical autoscaling: 
    # horizontal scaling means increase the number of pod.
    # vertical scaling means increase resource for each pod.

    

In [ ]:
#Note about StatefulSet important attributes:
    # because each pod deploy with statefulset have it own volume. we have to define VolumeMount and volume or volumeClaimTemplates
    # like Deployment, we have to set replica (how many pod do we want), template and container (what application we want to deploy)

#Note about Service in worker conifg:
    # service: define out how access pod. we have to define protocal (which protocal use to access pod), port (which port of service resource to access), selector (how to select pod which belong to that service)

#Note about PodDisruptionBudget:
    # definition: it define minimum of pod of a service. 
    # we have to define selector (how to select pods), minAvailable or maxUnavailable
#Note about 

In [ ]:
#Note about Kind:
    #definition: kind is a tools which allow us to run kubernetes with multiple nodes in your local computer using docker.

#Note about kubectl:
    #kubectl is a command line tools to control your kubernetes cluster

#Note about helm:
    #helm is a command line tools and software which allow us to define and deploy kubernetes objects which instructure as code

In [ ]:
# the person we need to follow in term of airflow and kubernetes is Marc Lamberti. github profile: https://github.com/marclamberti
# this note is follow this github repo: https://github.com/marclamberti/webinar-airflow-chart
#let try to deploy kubernetes with local kubernetes cluster with multiple nodes (3 nodes (1 control + 2 worker))
    #the first step is to create local kubernetes cluster with multiple node:
    #define the cluster with kind and then deploy cluster with command: kind create cluster --name $cluster_name --config $config_file_path
    #example: kind create cluster --name kind-cluster --config kind-cluster.yaml
    #please read the config file for more information about how to config a cluster in kind
    
    #after create cluster, we check information about cluster with command: kubectl cluster-info, and kubectl get nodes -o wide

    #after we have check cluster information let create namespace for kubernetes object management: kubectl create namespace $namespace
    #check the newly created namespace with command: kubectl get namespace

    #after create namespace for kubernetes object management. we add airflow official helm chart to our local repo with command: helm repo add [name] [url] [flags]
    #example: helm repo add apache-airflow https://airflow.apache.org
    #to get the latest version of apache-airflow helm chart use command: helm repo update
    #to check that your helm chart is add to your local repo use command: helm search repo $search_key

    #after we have add and update the helm chat and then check with command helm search. we will deploy the chart with helm command.
    #deploy helm chart with command: helm install $release_name $chart_name --namespace $namespace --debug
    #example: helm install airflow apache-airflow/airflow --namespace airflow --debug --timeout 10
    #uninstall syntax: helm uninstall $release_name -n $namespace

    #check to check your install or deployment use command: helm ls -n $namespace. 
    #helm ls will return deployment name, namespace and version. version allow us to quickly return to previous version if we have something wrong with the new version.

    #check the status of pods with command: kubectl get pods -n $namespace

    #check the log of pods use command: kubectl logs $pod_name -n $namespace

    #you want to acess the airflow web service from local host use forwar-port command
    #example: kubectl port-forward $resource $local_port:$service_port

    #note that: each pod, service or other kubernetes resource is jut a container running in kubernetes cluster.
    #port-forward command is to create a tunnel between local host port to the container port of the resource.

    #from the begining, we use offical helm chart which is own by helm. we need to get config file to do the config
    #the helm chart is design so that we have multiple chart components but only one central config  named values.yaml
    #get config file with command: helm show values apache-airflow/airflow > values.yaml

    #Note about helm show command. which allow us to show the content of the component of helm chart.
    #helm show chart
    #helm show values

    #let discuss about values.yaml file, which is the config file of airflow with helm.
    #airflowHome: is the home directory of airflow contaiern
    #defaultAirflowRepository: docker repository where airflow image is pull from
    #defaultAirflowTag: the tag which define which image to pull from repository.
    #airflowVersion: is the version of airflow in helm

    #after explore the values.yaml file, we will do the actual change the config
    #change defaultAirflowTag and airflowVersion to 2.1
    #add configMap to values.yaml.
    #define where to use configMap in config file with extraEnvFrom.
    
    #after define configMap we have to apply to kubernetes so that kubernets have that object.
    #command to apply a resource defintion: kubectl apply -f variables.yaml

    #after finish the config we update helm deployment with command: helm upgrade --install
    #example: helm upgrade --install airflow apache-airflow/airflow --debug -n airflow -f values.yaml

    #after we have succesful update your deployment then check at the front-end. and then check with pod container
    #to go to the specific pod use command: kubectl exec --stdin --tty $pod_name -n $namespace -- bin/bash . kubectl exec --stdin --tty airflow-worker-0 -n airflow -- /bin/bash
    #this is to check the config of configMap with environment variable

    #the next requirements is install python packages. the best approach is build custom image with dependencies and then use that image in kubernetes.
    #build image with command: docker build -t $image_name:tag $dockerfile_path
    #after build image, we need to load or register this image to kubernetes cluster in this case we use kind
    #command to load image to kind kubernetes cluster: kind load docker-image $image_name --name $cluster_name. 
    # example: kind load docker-image airflow-custom:1.0.0 --name kind-cluster

    #after solve the problem with install python package by build custom image. we will setup remote dag
    #the idea is that airflow will communicate with git repository using ssh private and public key.
    #step one: create your ssh private and public key with command: ssh key-gen
    #step two: to the your repository > setting > deployment key > add your public key to repok
    #step thre: enable gitSync and do the config with repo and subPath (the dag directory relative to root of repo). remember to set correct ssh address format
    #step four: create secrete in kubernete with command: kubectl create secret generic airflow-ssh-git-secret --from-file=gitSshKey=$path_to_private_key -n $namespace
    #example: kubectl create secret generic airflow-ssh-git-secret --from-file=gitSshKey=/root/.ssh/airflow -n airflow
    #step five: after create kubernete secret, we just config in values.yaml at sshKeySecret property
    #step six: update again with new config values.yaml with command: helm upgrade --install airflow apache-airflow/airflow -f values.yaml -n airflow --debug



In [ ]:
#Note about build custom airflow image with Dockerfile
#in this file we use command FROM, COPY, RUN
    #FROM $repository:tag
    #COPY $host_file $container_address
    #RUN command
#Note: . character means current direct, .. character means parent directory. In the context of container . character mean the home directory of container


